# Plotting Drake Passage Transport in ACCESS-OM3

This notebook uses the initial spinup simulation of ACCESS-OM3.

In [ ]:
#The following cell must be in all notebooks!

In [1]:
#parameters

### USER EDIT start
esm_file='/g/data/ol01/access-om3-output/access-om3-025/MC_25km_jra_ryf-1.0-beta/experiment_datastore.json'
esm_file='/g/data/ol01/outputs/access-om3-25km/MC_25km_jra_iaf-1.0-beta-5165c0f8/datastore.json'
dpi=300
### USER EDIT stop

import os
from matplotlib import rcParams
%matplotlib inline
rcParams['figure.dpi']= dpi

# needed for mkmd START 
notebook_name='not_using_mkfigs.sh'
notebooks_dir = os.path.join(f"/g/data/tm70/{os.environ['USER']}/access-om3-paper-1", "notebooks")
os.chdir(notebooks_dir)
print("Changed working directory to:", os.getcwd())
from mkfigs_configdoc import mkmd,get_notebook_name
notebook_name=get_notebook_name(notebook_name)
## example usage (note if the first argument to mkmd remains the same, you can insert multiple figures from the same notebook): 
#plt.savefig(notebooks_dir+'/mkmd/'+notebook_name[:-6]+'1.png', dpi=300, bbox_inches='tight')
#mkmd("Template notebook","`"+notebook_name+"`: Example figure of ACCESS-OM3 sea surface height (m).",os.path.basename(os.path.dirname(esm_file)),notebook_name[:-6]+'1.png',notebooks_dir+'/mkmd/')
#mkmd("Template notebook","`"+notebook_name+"`: Example figure of another type.",os.path.basename(os.path.dirname(esm_file)),notebook_name[:-6]+'1.png',notebooks_dir+'/mkmd/')
# needed for mkmd STOP

plotfolder=f"/g/data/{os.environ['PROJECT']}/{os.environ['USER']}/access-om3-paper-figs/"
os.makedirs(plotfolder, exist_ok=True)

 # a similar cell under this means it's being run in batch
print("ESM datastore path: ",esm_file)
print("Plot folder path: ",plotfolder)

Changed working directory to: /g/data/tm70/cyb561/access-om3-paper-1/notebooks
Notebook name is: not_using_mkfigs.sh
ESM datastore path:  /g/data/ol01/outputs/access-om3-25km/MC_25km_jra_iaf-1.0-beta-5165c0f8/datastore.json
Plot folder path:  /g/data/tm70/cyb561/access-om3-paper-figs/


In [2]:
import intake
from dask.distributed import Client
import matplotlib.pyplot as plt
import cmocean as cm
import cftime


In [3]:
client = Client(threads_per_worker=1)
print(client.dashboard_link)

/g/data/xp65/public/apps/med_conda/envs/analysis3_edge-25.09/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33053 instead
  warnings.warn(


/proxy/33053/status


## Load ACCESS-OM3 data from ESM datastore

In [4]:
#datastore_path = "/g/data/ol01/access-om3-output/access-om3-025/MC_25km_jra_ryf-1.0-beta/experiment_datastore.json"

datastore = intake.open_esm_datastore(
    esm_file,
    columns_with_iterables=[
        "variable",
        "variable_long_name",
        "variable_standard_name",
        "variable_cell_methods",
        "variable_units"
    ]
)

In [15]:
umo_2d = datastore.search(variable="umo_2d").to_dask(
    xarray_open_kwargs = dict(
        chunks={"time": -1},
        decode_timedelta=True
    )
)

/g/data/xp65/public/apps/med_conda/envs/analysis3_edge-25.09/lib/python3.11/site-packages/intake_esm/core.py:301: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  records = grouped.get_group(internal_key).to_dict(orient='records')
2026-02-05 16:35:09,949 - distributed.worker - ERROR - Compute Failed
Key:       _delayed_open_ds-f8094618-0441-4102-b09a-720cde28976b
State:     executing
Task:  <Task '_delayed_open_ds-f8094618-0441-4102-b09a-720cde28976b' _delayed_open_ds(..., ...)>
Exception: "ImportError('/g/data/xp65/public/apps/med_conda/envs/analysis3_edge-25.09/lib/python3.11/site-packages/netCDF4/../../../././libucc.so.1: undefined symbol: ucs_config_doc_nop')"
Traceback: '  File "/g/data/xp65/public/apps/med_conda/envs/analysis3_edge-25.09/lib/python3.11/site-packages/intake_esm/source.py", line 60, in _delayed_open_ds\n    return _op

ESMDataSourceError: Failed to load dataset with key='ocean.1mon.nv:2.xq:1441.yh:1152'
                 You can use `cat['ocean.1mon.nv:2.xq:1441.yh:1152'].df` to inspect the assets/files for this key.
                 

xp65/public/apps/med_conda/envs/analysis3_edge-25.09/lib/python3.11/site-packages/xarray/backends/netCDF4_.py", line 419, in open\n    import netCDF4\n  File "/g/data/xp65/public/apps/med_conda/envs/analysis3_edge-25.09/lib/python3.11/site-packages/netCDF4/__init__.py", line 3, in <module>\n    from ._netCDF4 import *\n'

2026-02-05 16:35:09,988 - distributed.worker - ERROR - Compute Failed
Key:       _delayed_open_ds-5c4ffb2c-8baf-41de-8604-6903683477a8
State:     executing
Task:  <Task '_delayed_open_ds-5c4ffb2c-8baf-41de-8604-6903683477a8' _delayed_open_ds(..., ...)>
Exception: "ImportError('/g/data/xp65/public/apps/med_conda/envs/analysis3_edge-25.09/lib/python3.11/site-packages/netCDF4/../../../././libucc.so.1: undefined symbol: ucs_config_doc_nop')"
Traceback: '  File "/g/data/xp65/public/apps/med_conda/envs/analysis3_edge-25.09/lib/python3.11/site-packages/intake_esm/source.py", line 60, in _delayed_open_ds\n    return _open_dataset(*args, **kwargs)\n           ^^^^^^^^^^^^^^^^^

## Load ACCESS-OM2 data from ACCESS-NRI Catalog

In [ ]:
catalog = intake.cat.access_nri

In [ ]:
var = catalog['025deg_jra55_ryf9091_gadi'].search(variable='tx_trans_int_z',frequency="1mon")
var.esmcat.aggregation_control.groupby_attrs = ['frequency']  ## Not sure why I have to do this!!
tx_trans_int_z=var.to_dask(
            xarray_open_kwargs=dict(
            chunks={"time": -1},
            decode_timedelta=True,
            use_cftime=True)
)

## Just to show the latitude I've picked ...

In [ ]:
umo_2d["umo_2d"].isel(time=0).plot()
plt.plot([-70,-70],[-71,-54])
plt.xlim([-100,-50])
plt.ylim([-80,-40])

## Timeseries of DP Transport at -70°

In [ ]:
DP_Trans_OM3 = umo_2d["umo_2d"].sel(xq=-70,method='nearest').sel(yh=slice(-71,-54)).sum("yh").load() / 1025 / 1e6

In [ ]:
DP_Trans_OM2 = tx_trans_int_z["tx_trans_int_z"].sel(xu_ocean=-70,method='nearest').sel(yt_ocean=slice(-71,-54)).sum("yt_ocean").sel(time=slice(cftime.datetime(1900, 1, 1,calendar='noleap'), cftime.datetime(1950, 12, 31,calendar='noleap'))).load() / 1036 / 1e6

In [ ]:
plt.figure(figsize=(10, 5)) 

DP_Trans_OM2.rolling(time=12).mean().plot(label='ACCESS-OM2')
DP_Trans_OM3.rolling(time=12).mean().plot(label='ACCESS-OM3')

plt.plot([cftime.datetime(1900, 1, 1), cftime.datetime(1950, 12, 31)], [173, 173],
           'k--', label="Donohue et al. (2016)")

plt.ylabel('Streamfunction (Sv)')
plt.title('Drake Passage Transport')
plt.legend()

plt.savefig(notebooks_dir+'/mkmd/'+notebook_name[:-6]+'1.png', dpi=300, bbox_inches='tight')


In [ ]:



mkmd("Drake Passage Transport","`"+notebook_name+"`: Drake passage transport in ACCESS-OM3 and ACCESS-OM",os.path.basename(os.path.dirname(esm_file)),notebook_name[:-6]+'1.png',notebooks_dir+'/mkmd/')


In [11]:
client.close()